In [2]:
import torch
import torch.nn as nn

/home/xps/anaconda3/envs/deeplearning/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
a = torch.rand(32, 8)
b = torch.zeros([32, 8], dtype=torch.float)
print(a, b)
nn.BCEWithLogitsLoss()(a, b).shape

tensor([[0.7177, 0.4620, 0.7463, 0.2622, 0.3404, 0.8826, 0.1439, 0.5462],
        [0.6309, 0.6814, 0.1998, 0.2262, 0.9070, 0.0082, 0.0479, 0.7617],
        [0.5016, 0.3702, 0.6201, 0.5696, 0.2294, 0.9190, 0.0020, 0.7711],
        [0.1165, 0.2065, 0.0255, 0.6900, 0.1338, 0.0218, 0.5307, 0.6795],
        [0.1792, 0.9402, 0.2608, 0.8130, 0.7946, 0.9927, 0.6176, 0.7968],
        [0.8698, 0.3424, 0.8237, 0.9733, 0.1571, 0.9639, 0.0024, 0.3086],
        [0.2014, 0.7760, 0.9917, 0.5833, 0.8315, 0.6831, 0.0733, 0.6837],
        [0.6015, 0.7881, 0.8965, 0.4180, 0.1227, 0.3583, 0.2159, 0.5546],
        [0.3885, 0.2047, 0.6540, 0.3742, 0.6651, 0.4297, 0.1425, 0.9823],
        [0.5690, 0.4163, 0.9368, 0.1713, 0.9099, 0.6591, 0.5020, 0.6121],
        [0.5934, 0.7378, 0.4533, 0.8970, 0.3521, 0.1714, 0.7168, 0.9302],
        [0.4527, 0.9106, 0.7258, 0.1114, 0.1236, 0.0957, 0.8794, 0.8629],
        [0.8235, 0.0162, 0.3315, 0.8898, 0.7282, 0.2121, 0.5806, 0.4557],
        [0.2145, 0.0186, 0.5194, 0.247

torch.Size([])